In [1]:
# ใช้ตัดคำภาษาไทย
import deepcut

# ใช้งาน regex
import re

# จัดการเกี่ยวกับ array
import numpy as np

import random
from sklearn.model_selection import train_test_split
import numpy

# สำหรับทำ classify และทดสอบโมเดล
import nltk
from nltk import FreqDist, precision, recall, f_measure, NaiveBayesClassifier
from nltk.classify import apply_features
from nltk.classify import util

# สำหรับสร้างชุดข้อมูลสำหรับ train และ test เพื่อทดสอบประสิทธิภาพ
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import collections, itertools

Using TensorFlow backend.


In [2]:
data_pos = [line.strip() for line in open("sentiment-assignment/pos.txt", 'r', encoding="utf8")]
data_neg = [line.strip() for line in open("sentiment-assignment/neg.txt", 'r', encoding="utf8")]
data_neu = [line.strip() for line in open("sentiment-assignment/neu.txt", 'r', encoding="utf8")]

In [3]:
random.shuffle(data_pos)
random.shuffle(data_neg)
random.shuffle(data_neu)

In [4]:
data_pos[:10]

['รอไป เหมือน MAYBELLINE รุ่นนั้น ที่เฉดแบบนี้ เมิงลืมแน่ๆ เด๋วส่งให้ดู',
 'ต้องลองๆ',
 'เรานี่กินจนติดไม่รู้ตัว เบียร์ก็กินสิงห์ น้ำก็กินสิงห์ ตอนนี้อยากกินพรีเซนเตอร์อะ อิอิ',
 'ทำไงดี ไดอารี่ สีชมพู ซื้อไม่เป็น',
 'ลดน้ำหนักอยู่',
 'ตอนเย็นก้อได้',
 'ติดใจพี่ช้างมากกว่า ไม่เคยทำให้ผิดหวัง กินทีไรเมาทุกที😂😂รสชาติดีต่อใจ',
 'ขอบคุณพี่ ๆ ทุกท่าน ที่ช่วยเป็นกระบอกเสียงแทนพวกเรา ผู้ใช้บุหรี่ไฟฟ้าทุกคนครับ',
 'ใครว่า "ขายฝัน" #อนาคตใหม่ ของ #บัตรทอง "ต้องใช้เงินเท่าไหร่" แล้วเอา "เงินจากไหน" มาฟังกันชัดๆ จากปาก ธนาธร แห่ง #พรรคอนาคตใหม่\\7\U0001f9e1 แจงละเอียดยิบ ใครยังสงสัย ส่งให้ฟังกันไปเลย 😉👌',
 'ว่างมั้ย ไปกินบาบีก้อนกัน']

In [5]:
data_neg[:10]

['โคตรบิดเบือนเลย ห่าลาก',
 'B1 ไม่ชอบกินไอติมอะ B2',
 'ม่ายไหวๆจิงๆ 5555 เอาแอคคอร์ดที่ยอดขายเน่าในเมกา มาจับทุบหน้าทุบตูด ทำไมฮอนด้าออกแบบล้าหลังไงไม่รุ้ ชอบฮอนด้ามาตลอดนะ แต่มันไม่สวยจิงๆอ่ะแล้วแต่คนชอบละกันคับ',
 'คิดว่าแป้ง cute press marine magic (สีเบจอ่อน) เป็นการลงทุนที่คุ้มค่ากว่า skinfood peach cotton powder ค่ะ แป้งละ…',
 'รถสตาร์ทไม่ติดซะแล้ว แง้ อะไหล่มาสด้าก็แพงด้วย ขอให้เป็นที่แบต เพราะถ้าเป็นที่อย่างอื่นก็ขอลาตรงนี้ ....',
 'กาวเหนียวกางเกงในไม่ขาดหรอกค่ะแต่ผ้าอนามัยจะขาดติดกางเกงใน แต่ใช้แบบเต็มที่มันไม่ติดหรอก',
 'เทอลดหุ่นอยู่นะ',
 'ชาชีสเอ็มเคไม่อร่อยเลยว้อย',
 'ฝ่าลมหนาวมากินแมค',
 'เออแต่ก็จริง แสงโสมเหมือนแดกกาว']

In [6]:
data_neu[:10]

['รังแกคนจน คนข้าขาย โคโลราโด้ เอชอาร์วี เทรลเบลเซอร์ ผมคนรวย พนักงานออฟฟิศ นิสสัน ซันนี่',
 'อัลเมร่า ดาว 99 ส่งเดือนเท่าไหร่ ขอรายละเอียดค่ะ',
 'บุรีรัมย์ฮึดบุกแบ่งแต้มแชมป์ลีกมังกร! ประเดิมถ้วยเอเชีย บุรีรัมย์ ยูไนเต็ด ตัวแทนเพียงหนึ่งเดียวจากไทย ประเดิมการแข่งขันเอเอฟซี แชมเปียนส์ ลีก ด้วยการบุกไปแบ่งแต้มจากกว่างโจว เอเวอร์แกรนด์ มาได้อย่างสุดมัน โดยรูปเกมในครึ่งแรกเจ้าถิ่นทำได้ดีกว่า และได้ประตูออกนำไปก่อน ก่อนที่ครึ่งหลังปราสาทสายฟ้าจะแก้เกมมาสู้ได้อย่างสนุกก่อนจะมาได้ประตูตีเสมอ 1-1 จากศูนย์หน้าตัวใหม่ เอดการ์ ซิลวา ทำให้เกมที่เหลืออีกกว่าครึ่งชั่วโมง มังกรแดนใต้ เปิดหน้าหวังทวงประตูคืนแต่ก็ไม่ผ่านแนวรับเซาะกราวที่วันนี้เก็บลูกอันตรายได้หมด นอกจากนั้นยังมีโอกาสลุ้นทำประตูแซง แต่ก็ไม่สามารถส่งบอลเข้าสู่ก้นตาข่ายได้ หมดเวลาบุรีรัมย์ ยูไนเต็ด คว้า 1 แต้มกลับบ้านสำเร็จ โปรแกรมนัดต่อไปจะพบกับเจจู ยูไนเต็ด จากเกาหลี ในวันที่ 21 กุมภาพันธ์ เป็นกำลังใจให้ตัวแทนหนึ่งเดียวจากไทยกันต่อไป #BuriramUnited #Chang #มากกว่ากีฬาคือน้ำใจนักกีฬา #ThaiBevThaiTalent',
 'เบียร์ช้าง ขวดเดี่ยว ไม่ก็แพ็ค

In [7]:
import re
import string

In [8]:
def clean_msg(msg):
    
    # ลบ text ที่อยู่ในวงเล็บ <> ทั้งหมด
    msg = re.sub(r'<.*?>','', msg)
    
    # ลบ hashtag
    msg = re.sub(r'#','',msg)
    
    # remove 1-2 character that don't have any meaning
#     msg = re.sub('[A-Za-z0-9]','',msg)
    msg = re.sub('[0-9][0-9]','',msg)
    
    # ลบ เครื่องหมายคำพูด (punctuation)
    for c in string.punctuation:
        msg = re.sub(r'\{}'.format(c),'',msg)
    
    # ลบ separator เช่น \n \t
    msg = ' '.join(msg.split())
    
    # remove emoji and symbols
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
                           "]+", flags=re.UNICODE)
    msg = re.sub(emoji_pattern ,'',msg)
    
    return msg

In [9]:
clean_neg = [clean_msg(str(txt)) for txt in data_neg]
clean_pos = [clean_msg(str(txt)) for txt in data_pos]
clean_neu = [clean_msg(str(txt)) for txt in data_neu]

In [10]:
clean_pos[-5:-1]

['เมนูที่ชอบสุดๆ คือ หมูคูโรบูตะแผ่นบางๆจิ้มกับน้ำจิ้มสูตรเด็ด ทานคู่กะกล่ำปีหั่นฝอย อร่อยสุด ฟินสุดทั้งครอบครัว หมูคูโรบูตะ คัดสรรคัดที่อะไร ตับไตไส้พุง คัดสรรหมู9 คัดสรรเนื้อ9',
 'แท็กตั้ง9ครั้งเลขมงคลไปแดกกก',
 'นี้ไงที่ตามหา',
 'มีแอปเอมเคยัง']

In [11]:
clean_pos[:20]

['รอไป เหมือน MAYBELLINE รุ่นนั้น ที่เฉดแบบนี้ เมิงลืมแน่ๆ เด๋วส่งให้ดู',
 'ต้องลองๆ',
 'เรานี่กินจนติดไม่รู้ตัว เบียร์ก็กินสิงห์ น้ำก็กินสิงห์ ตอนนี้อยากกินพรีเซนเตอร์อะ อิอิ',
 'ทำไงดี ไดอารี่ สีชมพู ซื้อไม่เป็น',
 'ลดน้ำหนักอยู่',
 'ตอนเย็นก้อได้',
 'ติดใจพี่ช้างมากกว่า ไม่เคยทำให้ผิดหวัง กินทีไรเมาทุกทีรสชาติดีต่อใจ',
 'ขอบคุณพี่ ๆ ทุกท่าน ที่ช่วยเป็นกระบอกเสียงแทนพวกเรา ผู้ใช้บุหรี่ไฟฟ้าทุกคนครับ',
 'ใครว่า ขายฝัน อนาคตใหม่ ของ บัตรทอง ต้องใช้เงินเท่าไหร่ แล้วเอา เงินจากไหน มาฟังกันชัดๆ จากปาก ธนาธร แห่ง พรรคอนาคตใหม่7 แจงละเอียดยิบ ใครยังสงสัย ส่งให้ฟังกันไปเลย ',
 'ว่างมั้ย ไปกินบาบีก้อนกัน',
 'นังมิ้น',
 'เค้าอยากกิน ',
 'พนักงานจะด่าไหม',
 'น่ากินนน',
 'สลายต้นตอสิวอุดตัน ด้วยประสิทธิภาพของสารสกัดธรรมชาติ Mangosteen Extract ลดเลือนริ้วรอย รอยดำ รอยแดง จากสิว ด้วยสารสกัดธรรมชาติ Centella Asiatica Extract คุณสมบัติที่ดีต่อผิวอ่อนโยน ปลอดภัยกับทุกสภาพผิว ปราศจากสารที่ทำให้เกิดการระคายเคือง และเช็ดสิ่งสกปรกที่เจอระหว่างวันได้สะอาดหมดจด น้ำมัน แอลกอฮอล์ SLS สารกันเสียชนิด parabenแล

In [12]:
def remove_emptyword_from_list(the_list, val, label):
    return [(word, label) for word in the_list if word != val]

final_pos = remove_emptyword_from_list(clean_pos,'','pos')
final_neg = remove_emptyword_from_list(clean_neg,'','neg')
final_neu = remove_emptyword_from_list(clean_neu,'','neu')

In [13]:
final_pos[:10]

[('รอไป เหมือน MAYBELLINE รุ่นนั้น ที่เฉดแบบนี้ เมิงลืมแน่ๆ เด๋วส่งให้ดู',
  'pos'),
 ('ต้องลองๆ', 'pos'),
 ('เรานี่กินจนติดไม่รู้ตัว เบียร์ก็กินสิงห์ น้ำก็กินสิงห์ ตอนนี้อยากกินพรีเซนเตอร์อะ อิอิ',
  'pos'),
 ('ทำไงดี ไดอารี่ สีชมพู ซื้อไม่เป็น', 'pos'),
 ('ลดน้ำหนักอยู่', 'pos'),
 ('ตอนเย็นก้อได้', 'pos'),
 ('ติดใจพี่ช้างมากกว่า ไม่เคยทำให้ผิดหวัง กินทีไรเมาทุกทีรสชาติดีต่อใจ',
  'pos'),
 ('ขอบคุณพี่ ๆ ทุกท่าน ที่ช่วยเป็นกระบอกเสียงแทนพวกเรา ผู้ใช้บุหรี่ไฟฟ้าทุกคนครับ',
  'pos'),
 ('ใครว่า ขายฝัน อนาคตใหม่ ของ บัตรทอง ต้องใช้เงินเท่าไหร่ แล้วเอา เงินจากไหน มาฟังกันชัดๆ จากปาก ธนาธร แห่ง พรรคอนาคตใหม่7 แจงละเอียดยิบ ใครยังสงสัย ส่งให้ฟังกันไปเลย ',
  'pos'),
 ('ว่างมั้ย ไปกินบาบีก้อนกัน', 'pos')]

In [14]:
final_neg[:10]

[('โคตรบิดเบือนเลย ห่าลาก', 'neg'),
 ('B1 ไม่ชอบกินไอติมอะ B2', 'neg'),
 ('ม่ายไหวๆจิงๆ เอาแอคคอร์ดที่ยอดขายเน่าในเมกา มาจับทุบหน้าทุบตูด ทำไมฮอนด้าออกแบบล้าหลังไงไม่รุ้ ชอบฮอนด้ามาตลอดนะ แต่มันไม่สวยจิงๆอ่ะแล้วแต่คนชอบละกันคับ',
  'neg'),
 ('คิดว่าแป้ง cute press marine magic สีเบจอ่อน เป็นการลงทุนที่คุ้มค่ากว่า skinfood peach cotton powder ค่ะ แป้งละ…',
  'neg'),
 ('รถสตาร์ทไม่ติดซะแล้ว แง้ อะไหล่มาสด้าก็แพงด้วย ขอให้เป็นที่แบต เพราะถ้าเป็นที่อย่างอื่นก็ขอลาตรงนี้',
  'neg'),
 ('กาวเหนียวกางเกงในไม่ขาดหรอกค่ะแต่ผ้าอนามัยจะขาดติดกางเกงใน แต่ใช้แบบเต็มที่มันไม่ติดหรอก',
  'neg'),
 ('เทอลดหุ่นอยู่นะ', 'neg'),
 ('ชาชีสเอ็มเคไม่อร่อยเลยว้อย', 'neg'),
 ('ฝ่าลมหนาวมากินแมค', 'neg'),
 ('เออแต่ก็จริง แสงโสมเหมือนแดกกาว', 'neg')]

In [15]:
final_neu[:10]

[('รังแกคนจน คนข้าขาย โคโลราโด้ เอชอาร์วี เทรลเบลเซอร์ ผมคนรวย พนักงานออฟฟิศ นิสสัน ซันนี่',
  'neu'),
 ('อัลเมร่า ดาว ส่งเดือนเท่าไหร่ ขอรายละเอียดค่ะ', 'neu'),
 ('บุรีรัมย์ฮึดบุกแบ่งแต้มแชมป์ลีกมังกร ประเดิมถ้วยเอเชีย บุรีรัมย์ ยูไนเต็ด ตัวแทนเพียงหนึ่งเดียวจากไทย ประเดิมการแข่งขันเอเอฟซี แชมเปียนส์ ลีก ด้วยการบุกไปแบ่งแต้มจากกว่างโจว เอเวอร์แกรนด์ มาได้อย่างสุดมัน โดยรูปเกมในครึ่งแรกเจ้าถิ่นทำได้ดีกว่า และได้ประตูออกนำไปก่อน ก่อนที่ครึ่งหลังปราสาทสายฟ้าจะแก้เกมมาสู้ได้อย่างสนุกก่อนจะมาได้ประตูตีเสมอ 11 จากศูนย์หน้าตัวใหม่ เอดการ์ ซิลวา ทำให้เกมที่เหลืออีกกว่าครึ่งชั่วโมง มังกรแดนใต้ เปิดหน้าหวังทวงประตูคืนแต่ก็ไม่ผ่านแนวรับเซาะกราวที่วันนี้เก็บลูกอันตรายได้หมด นอกจากนั้นยังมีโอกาสลุ้นทำประตูแซง แต่ก็ไม่สามารถส่งบอลเข้าสู่ก้นตาข่ายได้ หมดเวลาบุรีรัมย์ ยูไนเต็ด คว้า 1 แต้มกลับบ้านสำเร็จ โปรแกรมนัดต่อไปจะพบกับเจจู ยูไนเต็ด จากเกาหลี ในวันที่ กุมภาพันธ์ เป็นกำลังใจให้ตัวแทนหนึ่งเดียวจากไทยกันต่อไป BuriramUnited Chang มากกว่ากีฬาคือน้ำใจนักกีฬา ThaiBevThaiTalent',
  'neu'),
 ('เบียร์ช้าง

In [16]:
text = final_pos+final_neg+final_neu

In [18]:
len(text)

27264

In [17]:
train, test = train_test_split(text,test_size=0.15)

In [54]:
len(train)

23174

In [16]:
pos_train ,pos_test = train_test_split(final_pos,test_size=0.15)
neg_train ,neg_test = train_test_split(final_neg,test_size=0.15)
neu_train ,neu_test = train_test_split(final_neu,test_size=0.15)

In [17]:
len(pos_train)

4096

In [18]:
len(pos_test)

723

In [19]:
len(final_pos)

4819

In [20]:
len(neg_train)

5638

In [21]:
len(neg_test)

995

In [23]:
len(final_neg)

6633

In [18]:
def split_words (sentence):
    return deepcut.tokenize(''.join(sentence.lower().split()))
# sentences = [(split_words(sentence), sentiment) for (sentence, sentiment) in pos_train + neg_train + neu_train]

In [24]:
def get_words_in_sentences(sentences):
    all_words = []
    for (words, sentiment) in sentences:
        all_words.extend(words)
    return all_words

In [25]:
def get_word_features(list_of_words):
    list_of_words = nltk.FreqDist(list_of_words)
    word_features = list_of_words.keys()
    return word_features

In [26]:
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [ ]:
# all_reviews = []
# for (review, sentiment) in pos_reviews + neg_reviews:
#     reviews_filtered = [w.lower() for w in word_tokenize(review)]
#     all_reviews.append((reviews_filtered, sentiment))
# global features
# features = get_word_features(get_words_in_reviews(all_reviews))
# training_set = nltk.classify.apply_features(extract_features, all_reviews)
# classifier = nltk.NaiveBayesClassifier.train(training_set) # ทำการ train


In [19]:
train_text = [(split_words(sentence), sentiment) for (sentence, sentiment) in train]

In [28]:
len(sentences)

23174

In [29]:
import pandas
# create three rows of [0, 1, 2]
df = pandas.DataFrame(train_text)
my_columns = ["text", "label"]
df.columns = my_columns

In [30]:
df.head(5)

,text,label
0,"[ผ้า, เปิด, พอ, ได้, เห็น, กระจัง, หน้า, นิด, ...",neu
1,"[ยัง, ไง, ไทย, ก็, ได้, ใช้, และ, เป็น, ฐาน, ผ...",neu
2,"[วัน, ที่, ว่าง, น๊าาา, มา, เลยย, ๆ, ๆ]",pos
3,"[บาบีก้อน, กัน, งี้]",neu
4,[singhaisthebestbeer],pos


In [31]:
X = df.text
y = df.label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state = 42)

In [1]:
features_data = np.array(train_text[:5000])

NameError: name 'np' is not defined

In [34]:
X_train.head(5)

4059     [เบียร์, ใน, คอนไฮนเนเก้น, แก้ว, ละ, 0, บาท, แ...
18921                                    [ไป, ยัง, จ๊ะแพร]
16304    [มี, อะไร, ใ้ห้เรา, กู้, ภัย, ไหม, คะ, waterzo...
16559      [ป่วน, ไป, ทั่ว, เลย, นะ, ตัว, แสบ, ของ, วงการ]
12635                             [โฆษณา, drjill, อีก, ละ]
Name: text, dtype: object

In [47]:
X.head(10)

0    [ผ้า, เปิด, พอ, ได้, เห็น, กระจัง, หน้า, นิด, ...
1    [ยัง, ไง, ไทย, ก็, ได้, ใช้, และ, เป็น, ฐาน, ผ...
2              [วัน, ที่, ว่าง, น๊าาา, มา, เลยย, ๆ, ๆ]
3                                 [บาบีก้อน, กัน, งี้]
4                                [singhaisthebestbeer]
5                     [จศแถม, ถึง, ไม่, ทัน, มะว้าาาา]
6                                      [มื้อ, ใด, ล่ะ]
7    [ไป, สาด, น้ำ, กับข้า, เถิด, หนา, ออ, เจ้าข้า,...
8                                            [พระราม9]
9    [มิตซู, ผม, รับ, ประกัน, ห้า, ปี, ฟรี, ค่า, แร...
Name: text, dtype: object

In [53]:
# แบ่งข้อมูลเป็น 10 ชุด โดยไม่เรียง
k_fold = KFold(n_splits=10, random_state=456, shuffle=True)
word_features = None
accuracy_scores = []
for train_set, test_set in k_fold.split(features_data):
    word_features = get_word_features(get_words_in_sentences(features_data[train_set].tolist()))
    train_features = apply_features(extract_features, features_data[train_set].tolist())
    test_features = apply_features(extract_features, features_data[test_set].tolist())
    classifier = NaiveBayesClassifier.train(train_features)
    refsets = collections.defaultdict(set)
    testsets = collections.defaultdict(set)
    print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, test_features))*100)
#     for i, (feats, label) in enumerate(test_features):
#         refsets[label].add(i)
#         observed = classifier.classify(feats)
#         testsets[observed].add(i)
#     accuracy_scores.append(util.accuracy(classifier, test_features))
#     print('train: {} test: {}'.format(len(train_set), len(test_set)))
#     print('=================== Results ===================')
#     print('Accuracy {:f}'.format(accuracy_scores[-1]))
#     print('            Positive     Negative')
#     print('F1         [{:f}     {:f}]'.format(
#         f_measure(refsets['pos'], testsets['pos']),
#         f_measure(refsets['neg'], testsets['neg'])
#     ))
#     print('Precision  [{:f}     {:f}]'.format(
#         precision(refsets['pos'], testsets['pos']),
#         precision(refsets['neg'], testsets['neg'])
#     ))
#     print('Recall     [{:f}     {:f}]'.format(
#         recall(refsets['pos'], testsets['pos']),
#         recall(refsets['neg'], testsets['neg'])
#     ))
    print('===============================================\n')

Classifier accuracy percent: 65.4

Classifier accuracy percent: 68.4

Classifier accuracy percent: 68.2

Classifier accuracy percent: 67.9

Classifier accuracy percent: 70.1



MemoryError: 

In [ ]:
X_train

In [42]:
classifier = nltk.NaiveBayesClassifier.train(X_train)

print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, X_test))*100)

classifier.show_most_informative_features(15)

ValueError: too many values to unpack (expected 2)

In [58]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt

In [70]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

nb = Pipeline([
               
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

%%time
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

ValueError: setting an array element with a sequence.

In [82]:
author_name = {'pos':0,'neg':1,'nue':2}
y = train_text[:100]['label'].map(author_name) 

TypeError: list indices must be integers or slices, not str

In [81]:
mNB = MultinomialNB()

kf = KFold(n_splits=10,shuffle=True,random_state=49)
pred_test_full = 0
cv_score = []
i=1
for train_index,test_index in kf.split(features_data):
    print('{} of KFlod {}'.format(i,kf.n_splits))    
    xtr,xvl = features_data[train_index], features_data[test_index]
    ytr,yvl = y[train_index], y[test_index]
    
    mNB.fit(xtr,ytr)
    y_mNB = mNB.predict(xvl)
    cv_score.append(log_loss(yvl,mNB.predict_proba(xvl)))    
    pred_test_full += mNB.predict_proba(X_test_cv)
    i+=1
#roc_auc_score(yvl,mNB.predict_proba(xvl)[:,1]) # not for multi class
print(cv_score)
print('Mean accuracy score',np.mean(cv_score))
print('confusion matrix:\n',confusion_matrix(yvl,y_mNB))
del xtr,ytr,xvl,yvl

1 of KFlod 10


NameError: name 'X_cv' is not defined